In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

# Split it into a training set and a test set:

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SCALING THE DATA 

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# simple LinearSVR 

In [4]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

E:\ANAXONDA\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVR(random_state=42)

# Model performance on the training set:

In [6]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

# Root Mean Squarred Error ( RMSE:)

In [7]:
np.sqrt(mse)

0.9819256687727764

# CONCLUSION

In this training set, the targets are tens of thousands of dollars. The RMSE gives a rough idea of the kind of error you should expect (with a higher weight for large errors): so with this model we can expect errors somewhere around $10,000. Not great.

# method 2: RBF Kernel

In [8]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

In [9]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  25.8s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  25.7s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  33.2s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  25.3s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  25.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  25.1s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  23.7s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  28.7s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  28.2s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=  23.5s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=  23.3s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020CB59554C0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020CB5955790>},
                   random_state=42, verbose=2)

In [10]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

# Model evaluation

In [11]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785369

 Looks much better than the linear model. Let's select this model and evaluate it on the test set:

In [12]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.5929168385528745